# 概述
在这个文档中，我将使用我们自己实现的TextGenerate文本生成模型演示不同的解码策略带来的文本生成质量的影响,主要包括:<br>
`贪心搜索`<br>
`概率采样`<br>
`束搜索`<br>
`Top-K 采样`<br>
`Top-P 采样`<br>

#  模型定义与训练


## 模型定义
后面的解码策略中，我们都使用以下训练配置训练出来的模型进行不同的解码策略的演示

In [ ]:
import torch
from torch import nn


# 位置编码与嵌入
class EmbeddingPositionEncode(nn.Module):
    def __init__(self, d_model, dropout: float, vocab_size):
        super().__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, input_tensor: torch.Tensor) -> torch.Tensor:
        # x_1:(batch_size,seq_len,d_model)
        x_1 = self.embedding(input_tensor)
        seq_len = input_tensor.shape[1]

        # 创建位置编码(正余弦)
        position = torch.arange(seq_len, device=input_tensor.device).unsqueeze(
            1
        )  # unsqueeze(1)添加批次维度
        div_term = torch.exp(
            torch.arange(0, self.d_model, 2, device=input_tensor.device)
            * (
                    -torch.log(torch.tensor(10000.0, device=input_tensor.device))
                    / self.d_model
            )
        )

        pos_encoding = torch.zeros(seq_len, self.d_model, device=input_tensor.device)
        pos_encoding[:, 0::2] = torch.sin(position * div_term)
        pos_encoding[:, 1::2] = torch.cos(position * div_term)

        # 添加位置编码
        x_2 = pos_encoding.unsqueeze(0)
        return self.dropout(x_1 + x_2)


# 多头注意力
class MultiHeadAttention(nn.Module):

    def __init__(
            self, d_model: int, heads: int, dropout: float = 0, mask: bool = False
    ):

        super().__init__()
        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)  # 变换回 d_model

        self.d_model = d_model
        self.mask = mask

        self.heads = heads

        self.head_dim = d_model // heads
        self.dropout = nn.Dropout(dropout)

    def forward(
            self,
            query: torch.Tensor,
            key: torch.Tensor,
            value: torch.Tensor,
            key_padding_mask: torch.Tensor = None,
    ) -> torch.Tensor:
        # (batch_size,seq_len,d_model)
        batch_size, seq_len_q = query.size(0), query.size(1)
        seq_len_k = key.size(1)
        # 线性投影,分割多头
        # (batch_size,heads,seq_len_q,head_dim)
        q = (
            self.W_q(query)
            .view(batch_size, seq_len_q, self.heads, self.head_dim)
            .transpose(1, 2)
        )
        # (batch_size,heads,seq_len_k,head_dim)
        k = (
            self.W_k(key)
            .view(batch_size, seq_len_k, self.heads, self.head_dim)
            .transpose(1, 2)
        )
        # (batch_size,heads,seq_len_k,head_dim)
        v = (
            self.W_v(value)
            .view(batch_size, seq_len_k, self.heads, self.head_dim)
            .transpose(1, 2)
        )

        # 计算注意力分数
        # scores:(batch_size,heads,seq_len_q,seq_len_k)
        scores = q @ k.transpose(-2, -1)
        # 因果掩码
        if self.mask:
            mask_matrix = torch.triu(
                torch.full((seq_len_q, seq_len_k), float("-inf")), diagonal=1
            ).to(query.device)
            scores = scores + mask_matrix
        # 掩蔽字符<pad>
        if key_padding_mask is not None:
            # 确保key_padding_mask是布尔类型
            if key_padding_mask.dtype != torch.bool:
                key_padding_mask = key_padding_mask.bool()

            # 原始形状: (batch_size, seq_len_k)
            # 目标形状: (batch_size, 1, 1, seq_len_k) 
            key_padding_mask = key_padding_mask.unsqueeze(1).unsqueeze(1)

            # 应用掩码
            scores = scores.masked_fill(key_padding_mask, -1e9)
        # 缩放并应用softmax
        attention = nn.Softmax(dim=-1)(
            scores / torch.sqrt(torch.tensor(self.head_dim, device=query.device))
        )
        # attention:(batch_size,heads,seq_len_q,seq_len_k)
        attention = self.dropout(attention)
        # 加权和
        # out:(batch_size,heads,seq_len_q,head_dim)
        out = attention @ v
        # 拼接多头
        out = (
            out.transpose(1, 2)
            .contiguous()
            .view(batch_size, seq_len_q, self.heads * self.head_dim)
        )
        # (batch_size,seq_len_q,d_model)
        return self.W_o(out)


# 解码器层
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout):
        super().__init__()
        self.multi_head_attention = MultiHeadAttention(
            d_model, heads, dropout, mask=True
        )
        self.layer_norm_1 = nn.LayerNorm(d_model)
        self.layer_norm_2 = nn.LayerNorm(d_model)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, d_model * 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(d_model * 4, d_model),
        )
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, key_padding_mask):
        # 自注意力子层
        residual = x
        x = self.multi_head_attention(x, x, x, key_padding_mask)
        x = self.layer_norm_1(residual + self.dropout(x))

        # 前馈子层
        residual = x
        x = self.feed_forward(x)
        x = self.layer_norm_2(residual + self.dropout(x))

        return x


class TextGenerate(nn.Module):
    def __init__(self, d_model, vocab_size, num_layers=6, heads=8, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.embedding_pos_encode = EmbeddingPositionEncode(
            d_model, dropout, vocab_size
        )

        # 堆叠多层解码器
        self.layers = nn.ModuleList(
            [DecoderLayer(d_model, heads, dropout) for _ in range(num_layers)]
        )

        self.final_linear = nn.Linear(d_model, vocab_size)

    def forward(self, x, key_padding_mask):
        x = self.embedding_pos_encode(x)

        for layer in self.layers:
            x = layer(x, key_padding_mask)

        return self.final_linear(x)

## 训练

In [ ]:
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from LyricsDataset import LyricsDataset
from tqdm import tqdm
from torch.utils.data import random_split


def save_checkpoint(epoch, model, optimizer, scheduler, loss, path):
    checkpoint = {
        "epoch": epoch,
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state": scheduler.state_dict(),
        "scheduler_type": type(scheduler).__name__,
        "loss": loss,
    }
    torch.save(checkpoint, path)


def load_checkpoint(model, optimizer, scheduler, path):
    if path is not None:
        checkpoint = torch.load(path)
        if model:
            model.load_state_dict(checkpoint["model_state_dict"])
        if optimizer:
            optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
            print(f"从{checkpoint['epoch']}开始训练")
        if scheduler:
            scheduler.load_state_dict(checkpoint["scheduler_state"])
        return checkpoint["epoch"], checkpoint["loss"]

    print("未发现检查点")
    return 0, float("inf")


device = "cuda" if torch.cuda.is_available() else "cpu"

batch_size = 12
dataset = LyricsDataset("../data/generate/lyrics.csv", nrows=-1, batch_size=batch_size)
train_dataset, test_dataset = random_split(dataset, [0.9, 0.1])  # 百分之九十作为训练集

train_loader, test_loader = DataLoader(
    train_dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, pin_memory=True
), DataLoader(
    test_dataset, batch_size=batch_size, collate_fn=dataset.collate_fn, pin_memory=True
)

epochs = 500

d_model = 512
vocab_size = len(dataset.token_to_index)
heads = 8
num_layers = 6
dropout = 0.1
model = TextGenerate(
    d_model=d_model,
    vocab_size=vocab_size,
    num_layers=num_layers,
    heads=heads,
    dropout=dropout,
)

padding_idx = dataset.token_to_index["<pad>"]
loss_fn = nn.CrossEntropyLoss(ignore_index=padding_idx)

lr = 1.0
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=lr,
    betas=(0.9, 0.98),
    eps=1e-9,
    weight_decay=1e-4,
)

total_steps = int(epochs * (len(dataset) / batch_size))
warmup_steps = max(1, int(total_steps * 0.4))
scheduler = LambdaLR(
    optimizer=optimizer,
    lr_lambda=lambda step: (
            512 ** (-0.5)  # 模型维度的平方根倒数
            * min(
        (step + 1) ** (-0.5),  # 衰减阶段：步长的平方根倒数
        (step + 1) * (warmup_steps ** (-1.5)),  # 预热阶段：线性增长
    )
    ),
)
print(f"total_steps:{total_steps},warmup_steps:{warmup_steps}")


def evaluate(model, test_loader, device):
    model.eval().to(device)
    total_val_loss = 0
    with torch.no_grad():
        pbar = tqdm(test_loader, desc="eval progress ", leave=False)
        for src, tgt in pbar:
            src_key_padding_mask = src == padding_idx
            src_key_padding_mask = src == padding_idx
            src, tgt, src_key_padding_mask = (
                src.to(device),
                tgt.to(device),
                src_key_padding_mask.to(device),
            )
            pred = model(src, src_key_padding_mask)
            loss = loss_fn(pred.reshape(-1, vocab_size), tgt.reshape(-1))
            total_val_loss += loss.item()
    return total_val_loss / len(test_loader)


scaler = torch.amp.GradScaler(device)

error = []
path = None
best_val_loss = 1e10
start_epoch, loss = load_checkpoint(model, optimizer, scheduler, path)
for epoch in range(start_epoch, epochs):
    model.train().to(device)
    total_loss = 0
    pbar = tqdm(
        train_loader, desc=f"Epoch [{epoch + 1}/{epochs}] | Epoch progress", leave=False
    )
    for src, tgt in pbar:
        src_key_padding_mask = src == padding_idx
        src, tgt, src_key_padding_mask = (
            src.to(device),
            tgt.to(device),
            src_key_padding_mask.to(device),
        )
        optimizer.zero_grad()
        with torch.amp.autocast(device):
            pred = model(src, src_key_padding_mask)
            loss = loss_fn(pred.reshape(-1, vocab_size), tgt.reshape(-1))

        pbar.set_postfix(loss=f"{loss.item():.6f}")
        total_loss += loss.item()

        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

    val_loss = evaluate(model, test_loader, device)
    if val_loss < best_val_loss:
        torch.save(
            model.state_dict(), "../model/解码策略/best_lyrics_model.pth"
        )  # 保存最好模型
        best_val_loss = val_loss
    avg_loss = total_loss / len(train_loader)
    error.append(avg_loss)
    if (epoch + 1) % 1 == 0:
        print(
            f"epoch {epoch + 1}, loss: {avg_loss:.6f}, perplexity: {torch.exp(torch.tensor(avg_loss)).item():.6f},val_loss: {val_loss}"
        )
    if (epoch + 1) % 1 == 0:
        path_to_save = f"../checkpoints/epoch_{epoch + 1}.pth"
        save_checkpoint(epoch + 1, model, optimizer, scheduler, loss, path_to_save)
plt.style.use("ggplot")
plt.plot(error)
plt.xlabel("epoch")
plt.ylabel("loss")
plt.show()

# 加载模型

In [ ]:
from LyricsDataset import LyricsDataset

batch_size = 12
dataset = LyricsDataset("../data/generate/lyrics.csv", nrows=-1, batch_size=batch_size)
d_model = 512
vocab_size = len(dataset.token_to_index)
num_layers = 6
heads = 8
dropout = 0.1
model = TextGenerate(
    d_model=d_model,
    vocab_size=vocab_size,
    num_layers=num_layers,
    heads=heads,
    dropout=dropout,
)
model.load_state_dict(torch.load('../model/解码策略/best_lyrics_model.pth'))

# 贪心搜索

每一次从模型下一次的预测中选择概率最大的词作为输出

In [ ]:
def predict(
        text: str,
        model: nn.Module,
        max_length: int,
        separator: str,
        device: str,
        to_index,
        to_token,
        temperature: float = 0.75,
):
    model.eval().to(device)

    def generate(splitted_text):
        with torch.no_grad():
            index_text = [to_index['<bos>']] + [to_index[char] for char in splitted_text]  # 添加句首标记并将文本转化为索引
            tensor_text = torch.tensor(index_text, device=device).unsqueeze(0)
            generated = index_text.copy()
            for _ in range(max_length):
                # 自回归生成没有<pad>,因此mask传入None
                pred = model(tensor_text, None)[:, -1, :] / temperature  # 应用温度
                # 使用argmax贪心预测
                next_id = pred.argmax(dim=-1)
                # 添加新next_id到下一次的输入中
                tensor_text = torch.cat((tensor_text, next_id.unsqueeze(0)), dim=-1)
                if to_token[next_id.item()] == "<eos>":
                    break
                generated.append(next_id.item())
            return generated

    generate_text = []
    for splitted_text in text.split(
            separator
    ):  # 按照separator分割，分割后的每个元素作为每一句的开头
        generate_text += list(
            splitted_text
        )  # 将新的splitted_text转化为列表添加到generate_text中
        generate_text = [
            to_token[idx] for idx in generate(generate_text)
        ]  # 上一次的输出拼接上新加入的token作为输入，以实现上下文关联
        generate_text.append("，")  # 添加逗号

    return "".join(generate_text).strip("<bos>").replace("，，", "，")


text = "玫瑰/晚风"
generated_lyrics = predict(
    text,
    model,
    100,
    "/",
    "cuda",
    dataset.token_to_index,
    dataset.index_to_token,
    temperature=0.95,
)

generated_lyrics  # 生成内容重复

# 概率采样

对模型的预测输出进行归一化之后，如果概率 我:30% ,爱:30%, 你:40% ,表示有百分之三十的概率选中 我 这个词作为下一次的预测输出

In [ ]:
def predict(
        text: str,
        model: nn.Module,
        max_length: int,
        separator: str,
        device: str,
        to_index,
        to_token,
        temperature: float = 0.75,
):
    model.eval().to(device)

    def generate(splitted_text):
        with torch.no_grad():
            index_text = [to_index['<bos>']] + [to_index[char] for char in splitted_text]  # 添加句首标记并将文本转化为索引
            tensor_text = torch.tensor(index_text, device=device).unsqueeze(0)
            generated = index_text.copy()
            for _ in range(max_length):
                # 自回归生成没有<pad>,因此mask传入None
                pred = model(tensor_text, None)[:, -1, :] / temperature  # 应用温度
                # 概率采样预测
                proba = nn.Softmax(dim=-1)(pred)
                dist = torch.distributions.Categorical(proba)
                next_id = dist.sample()
                # 添加新next_id到下一次的输入中
                tensor_text = torch.cat((tensor_text, next_id.unsqueeze(0)), dim=-1)
                if to_token[next_id.item()] == "<eos>":
                    break
                generated.append(next_id.item())
            return generated

    generate_text = []
    for splitted_text in text.split(
            separator
    ):  # 按照separator分割，分割后的每个元素作为每一句的开头
        generate_text += list(
            splitted_text
        )  # 将新的splitted_text转化为列表添加到generate_text中
        generate_text = [
            to_token[idx] for idx in generate(generate_text)
        ]  # 上一次的输出拼接上新加入的token作为输入，以实现上下文关联
        generate_text.append("，")  # 添加逗号

    return "".join(generate_text).strip("<bos>").replace("，，", "，")


text = "玫瑰/晚风"
generated_lyrics = predict(
    text,
    model,
    100,
    "/",
    "cuda",
    dataset.token_to_index,
    dataset.index_to_token,
    temperature=0.95,
)

generated_lyrics

# 束搜索

束搜索是生成任务中一种平衡效率与质量的解码策略，它在每一步生成时，会从所有可能的候选词中筛选出概率最高的k个（k为束宽），形成当前的候选序列集合；接着，基于这些候选序列分别扩展下一个词，每个序列都会生成新的候选，随后从所有新生成的候选中再次筛选出概率最高的k个继续扩展，以此类推，直到达到最大长度或生成结束符。

In [ ]:
def beam_search(model, initial_tensor, k, to_token, max_length, device):
    # 初始化候选序列：(序列, 累积概率, 长度)
    candidates = [
        (initial_tensor, torch.tensor(0.0, device=device), 0)
    ]

    completed = []

    for _ in range(max_length):
        new_candidates = []

        # 扩展每个候选序列
        for seq, prob, length in candidates:
            # 如果序列已结束，直接添加到完成列表
            if length > 0 and to_token[seq[0, -1].item()] == '<eos>':
                completed.append((seq, prob))
                continue

            # 获取下一个词的预测
            pred = model(seq, None)[:, -1, :] 
            proba = nn.Softmax(dim=-1)(pred)

            # 获取topk个候选词
            top_probs, top_indices = proba.topk(k, dim=-1)
            top_probs = top_probs.squeeze(0)
            top_indices = top_indices.squeeze(0)

            # 扩展序列
            for i in range(k):
                idx = top_indices[i].unsqueeze(0).unsqueeze(0)
                new_seq = torch.cat([seq, idx], dim=-1)
                new_prob = prob + torch.log(top_probs[i])  # 使用对数概率避免下溢
                new_length = length + 1

                new_candidates.append((new_seq, new_prob, new_length))

        # 如果没有新候选，提前结束
        if not new_candidates:
            break

        # 按概率排序并保留topk个候选
        new_candidates.sort(key=lambda x: x[1], reverse=True)
        candidates = new_candidates[:k]

    # 将剩余未完成的候选添加到结果中
    completed.extend([(seq, prob) for seq, prob, _ in candidates])

    # 按概率排序并返回
    completed.sort(key=lambda x: x[1], reverse=True)
    return [(seq.reshape(-1).tolist(), prob) for seq, prob in completed]


def predict(
        text: str,
        model: nn.Module,
        max_length: int,
        separator: str,
        device: str,
        to_index,
        to_token,
        k: int = 5
):
    model.eval().to(device)

    def generate(splitted_text):
        with torch.no_grad():
            index_text = [to_index['<bos>']] + [to_index[char] for char in splitted_text]  # 添加句首标记并将文本转化为索引
            tensor_text = torch.tensor(index_text, device=device).unsqueeze(0)
            # 自回归生成没有<pad>,因此mask传入None
            generated = sorted(beam_search(model, tensor_text, k, to_token, max_length, device),
                               key=lambda x: x[1].item(),
                               reverse=True)[0][0]
            return generated

    generate_text = []
    for splitted_text in text.split(
            separator
    ):  # 按照separator分割，分割后的每个元素作为每一句的开头
        generate_text += list(
            splitted_text
        )  # 将新的splitted_text转化为列表添加到generate_text中
        generate_text = [
            to_token[idx] for idx in generate(generate_text)
        ]  # 上一次的输出拼接上新加入的token作为输入，以实现上下文关联
        generate_text.append("，")  # 添加逗号

    return "".join(generate_text).strip("<bos>").replace("，，", "，")


text = "玫瑰/晚风"
generated_lyrics = predict(
    text,
    model,
    100,
    "/",
    "cuda",
    dataset.token_to_index,
    dataset.index_to_token,
    k=10
)

generated_lyrics

# Top-K 采样

每一步生成下一个词时，先让模型预测出整个词汇表中所有词的概率分布，然后从中筛选出概率排名前 K 个的候选词（形成一个缩小的候选集合），接着对这 K 个候选词的概率进行重新归一化，最后按照归一化后的概率随机选择一个词作为当前步生成的词，再基于该词继续下一步生成，直到达到预设长度或生成结束符

In [ ]:
def predict(
        text: str,
        model: nn.Module,
        max_length: int,
        separator: str,
        device: str,
        to_index,
        to_token,
        temperature: float = 0.75,
        k: int = 5
):
    model.eval().to(device)

    def generate(splitted_text):
        with torch.no_grad():
            index_text = [to_index['<bos>']] + [to_index[char] for char in splitted_text]  # 添加句首标记并将文本转化为索引
            tensor_text = torch.tensor(index_text, device=device).unsqueeze(0)
            generated = index_text.copy()
            for _ in range(max_length):
                # 自回归生成没有<pad>,因此mask传入None
                pred = model(tensor_text, None)[:, -1, :] / temperature  # 应用温度
                proba = nn.Softmax(dim=-1)(pred).reshape(-1)
                # top-k采样
                proba_values, proba_indices = proba.topk(k)
                proba_values = nn.Softmax(dim=-1)(proba_values)
                dist = torch.distributions.Categorical(nn.Softmax(dim=-1)(proba_values)) # 概率采样
                next_id = proba_indices[dist.sample()].unsqueeze(0)
                # 添加新next_id到下一次的输入中
                tensor_text = torch.cat((tensor_text, next_id.unsqueeze(0)), dim=-1)
                if to_token[next_id.item()] == "<eos>":
                    break
                generated.append(next_id.item())
            return generated

    generate_text = []
    for splitted_text in text.split(
            separator
    ):  # 按照separator分割，分割后的每个元素作为每一句的开头
        generate_text += list(
            splitted_text
        )  # 将新的splitted_text转化为列表添加到generate_text中
        generate_text = [
            to_token[idx] for idx in generate(generate_text)
        ]  # 上一次的输出拼接上新加入的token作为输入，以实现上下文关联
        generate_text.append("，")  # 添加逗号

    return "".join(generate_text).strip("<bos>").replace("，，", "，")


text = "玫瑰/晚风"
generated_lyrics = predict(
    text,
    model,
    500,
    "/",
    "cuda",
    dataset.token_to_index,
    dataset.index_to_token,
    temperature=0.95,
    k=5
)

generated_lyrics

# Top-P 采样
每一步生成下一个词时，模型会先预测整个词汇表的概率分布并按概率从高到低排序，然后依次累加这些词的概率，直到累积概率达到预设的阈值 P，此时所有参与累积的词就构成了候选集合；之后对这个候选集合内词的概率进行重新归一化，确保它们的概率和为 1，最后从这个集合中随机采样一个词作为当前步生成的词，重复该过程直到生成结束符或达到最大长度

In [ ]:
def predict(
        text: str,
        model: nn.Module,
        max_length: int,
        separator: str,
        device: str,
        to_index,
        to_token,
        temperature: float = 0.75,
        p: float = 0.75
):
    model.eval().to(device)

    def generate(splitted_text):
        with torch.no_grad():
            index_text = [to_index['<bos>']] + [to_index[char] for char in splitted_text]  # 添加句首标记并将文本转化为索引
            tensor_text = torch.tensor(index_text, device=device).unsqueeze(0)
            generated = index_text.copy()
            for _ in range(max_length):
                # 自回归生成没有<pad>,因此mask传入None
                pred = model(tensor_text, None)[:, -1, :] / temperature  # 应用温度
                proba = nn.Softmax(dim=-1)(pred).reshape(-1)
                # top-p采样
                proba_values, proba_indices = proba.sort(descending=True)
                proba_cumsum = proba_values.cumsum(dim=-1)
                indices = proba_indices[proba_cumsum <= p]
                # 处理候选为空的情况
                if len(indices) == 0:
                    # 如果没有符合条件的候选，就选择概率最高的token
                    indices = proba_indices[:1]
                candidate_values = proba[indices]
                dist = torch.distributions.Categorical(nn.Softmax(dim=-1)(candidate_values))
                next_id = indices[dist.sample()].unsqueeze(0)
                # 添加新next_id到下一次的输入中
                tensor_text = torch.cat((tensor_text, next_id.unsqueeze(0)), dim=-1)
                if to_token[next_id.item()] == "<eos>":
                    break
                generated.append(next_id.item())
            return generated

    generate_text = []
    for split_text in text.split(
            separator
    ):  # 按照separator分割，分割后的每个元素作为每一句的开头
        generate_text += list(
            split_text
        )  # 将新的split_text转化为列表添加到generate_text中
        generate_text = [
            to_token[idx] for idx in generate(generate_text)
        ]  # 上一次的输出拼接上新加入的token作为输入，以实现上下文关联
        generate_text.append("，")  # 添加逗号

    return "".join(generate_text).strip("<bos>").replace("，，", "，")


text = "我吹过你吹过的晚风，"
generated_lyrics = predict(
    text,
    model,
    500,
    "/",
    "cuda",
    dataset.token_to_index,
    dataset.index_to_token,
    temperature=1.1,
    p=0.75
)

generated_lyrics  